In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [14]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [15]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [16]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [17]:
df_encoded = pd.get_dummies(df, columns=["home_ownership",
                                         "verification_status",
                                         "pymnt_plan",
                                         "application_type",
                                         "hardship_flag",
                                         "debt_settlement_flag",
                                         "initial_list_status",
                                         "next_pymnt_d"])

months_num = {
    'Jan-2019': 1,
    'Feb-2019': 2,
    'Mar-2019':3
}

# Months' and Loan Status condition encoded using the dictionary values
df_encoded["issue_d_num"] = df_encoded["issue_d"].apply(lambda x: months_num[x])
# Drop the issue_d columns
df_encoded = df_encoded.drop(["issue_d"], axis=1)

df_encoded.head(10)

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,pymnt_plan_n,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_num
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,1,0,1,0,1,3
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,1,0,1,0,1,3
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,1,0,1,0,1,3
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,1,0,1,0,1,3
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,1,0,1,0,1,3
5,25000.0,0.1797,634.43,90000.0,low_risk,18.49,1.0,1.0,14.0,0.0,...,1,1,0,1,1,0,1,0,1,3
6,20400.0,0.2000,540.48,51315.0,low_risk,25.63,0.0,2.0,8.0,1.0,...,1,1,0,1,1,0,1,0,1,3
7,10000.0,0.1557,349.46,66000.0,low_risk,9.00,0.0,0.0,11.0,0.0,...,1,1,0,1,1,0,1,0,1,3
8,14000.0,0.0881,443.96,45000.0,low_risk,10.24,0.0,1.0,9.0,0.0,...,1,1,0,1,1,1,0,0,1,3
9,10000.0,0.1474,345.39,63295.0,low_risk,39.34,0.0,0.0,27.0,0.0,...,1,1,0,1,1,0,1,0,1,3


In [18]:
set(df_encoded.columns) - set(df.columns)

{'application_type_Individual',
 'application_type_Joint App',
 'debt_settlement_flag_N',
 'hardship_flag_N',
 'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'initial_list_status_f',
 'initial_list_status_w',
 'issue_d_num',
 'next_pymnt_d_Apr-2019',
 'next_pymnt_d_May-2019',
 'pymnt_plan_n',
 'verification_status_Not Verified',
 'verification_status_Source Verified',
 'verification_status_Verified'}

# Split the Data into Training and Testing

In [19]:
# Create our features
X = df_encoded.drop("loan_status", axis=1)

# Create our target
y = df_encoded[["loan_status"]]

In [20]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,pymnt_plan_n,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_num
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.0,0.860340,0.139660,1.0,1.0,0.123879,0.876121,0.383161,0.616839,1.726172
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.346637,0.346637,0.0,0.0,0.329446,0.329446,0.486161,0.486161,0.743862
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000,0.000000,1.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.0,1.000000,0.000000,1.0,1.0,0.000000,1.000000,0.000000,0.000000,1.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.0,1.000000,0.000000,1.0,1.0,0.000000,1.000000,0.000000,1.000000,2.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.0,1.000000,0.000000,1.0,1.0,0.000000,1.000000,1.000000,1.000000,2.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,3.000000


In [21]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'loan_status': 1})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [24]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

clf = BalancedRandomForestClassifier(random_state=0, n_estimators = 100)
clf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=0)

In [25]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = clf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7646711671868778

In [26]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   66,    35],
       [ 2123, 14981]], dtype=int64)

In [27]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.65      0.88      0.06      0.76      0.56       101
   low_risk       1.00      0.88      0.65      0.93      0.76      0.59     17104

avg / total       0.99      0.87      0.65      0.93      0.76      0.58     17205



In [36]:
# List the features sorted in descending order by feature importance
sorted(zip(X_train.columns, clf.feature_importances_), reverse=True, key=lambda k: k[1])

[('last_pymnt_amnt', 0.06811790579118143),
 ('total_rec_int', 0.06274685997494389),
 ('total_rec_prncp', 0.060983010800181287),
 ('total_pymnt', 0.05835592622307053),
 ('total_pymnt_inv', 0.05413080080062401),
 ('issue_d_num', 0.030573316832865606),
 ('int_rate', 0.03028940073855276),
 ('dti', 0.020011594261998496),
 ('total_bc_limit', 0.01774166370026818),
 ('revol_bal', 0.01728304309957303),
 ('installment', 0.016475216227421593),
 ('max_bal_bc', 0.016316338234743367),
 ('mths_since_recent_inq', 0.016274408560609837),
 ('tot_hi_cred_lim', 0.015834878460514058),
 ('out_prncp_inv', 0.015730198136482826),
 ('out_prncp', 0.015530256437171281),
 ('annual_inc', 0.015307389541346209),
 ('all_util', 0.014578624231293767),
 ('mths_since_rcnt_il', 0.014550759353995295),
 ('tot_cur_bal', 0.014222068372674116),
 ('mo_sin_old_il_acct', 0.01383797859564329),
 ('loan_amnt', 0.01354312895861414),
 ('total_bal_il', 0.013494532632962386),
 ('avg_cur_bal', 0.0134140046225337),
 ('total_rev_hi_lim', 0.0

### Easy Ensemble AdaBoost Classifier

In [37]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=1)
eec.fit(X_train, y_train) 

EasyEnsembleClassifier(random_state=1)

In [41]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9197038038696292

In [42]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   93,     8],
       [ 1392, 15712]], dtype=int64)

In [43]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.06      0.92      0.92      0.12      0.92      0.85       101
   low_risk       1.00      0.92      0.92      0.96      0.92      0.85     17104

avg / total       0.99      0.92      0.92      0.95      0.92      0.85     17205

